In [91]:
import boto3
import csv
import json
import os
import requests
from io import BytesIO;
from os import listdir
from os.path import isfile, join

In [6]:
path = "/home/propietari/Documents/fitxers importants/WRI/Scraping_results/"

onlyfiles = [f for f in listdir(path) if isfile(join(path, f))]


In [7]:
onlyfiles

['ElSalvador_20210730.csv',
 'USFR_2_20210311.csv',
 'USFR_20210311.csv',
 'USFR_1_20210311.csv',
 'USFR_20210702.csv',
 'LeyChile_20210702.csv',
 'India_20210310.csv',
 'Mexico_20210703.csv',
 'USFR_20210310.csv',
 'India_20210703.csv',
 'El_Salvador_no_duplicates.csv']

In [75]:
# path = 'C:/Users/jordi/Documents/claus/'
path = '/home/propietari/Documents/claus/'
filename = 'AWS_S3_keys_wri.json'
file = path + filename
with open(file, 'r') as dict:
    credentials = json.load(dict)
                                      
KEY = list(credentials)[0]
SECRET = list(credentials.values())[0]
# s3BucketName = "wri-testing"
s3BucketName = "wri-nlp-policy"
# region = 'eu-central-1'
region = "us-east-1"

s3 = boto3.resource(
    service_name = 's3',
    region_name = region,
    aws_access_key_id = KEY,
    aws_secret_access_key = SECRET
)


### El Salvador

In [48]:
# Loading the csv files from scraping to be compared
file_position = 0
with open(path + onlyfiles[file_position], 'r', encoding = "Latin1") as f:
    reader = csv.reader(f)
    ElSalvador_20210730 = [row[3] for row in reader]

file_position = 10
with open(path + onlyfiles[file_position], 'r', encoding = "Latin1") as f:
    reader = csv.reader(f)
    El_Salvador_no_duplicates = [row[7] for row in reader]

In [53]:
ElSalvador_20210730_SET = set(ElSalvador_20210730)
print(f"There are {len(El_Salvador_no_duplicates)} items in the old file and {len(ElSalvador_20210730)} in the new one")
print(f"There are {len(ElSalvador_20210730) - len(ElSalvador_20210730_SET)} duplicates in the new version")
print(f"The difference between the first scraping and the last version is of {len(El_Salvador_no_duplicates) - len(ElSalvador_20210730)} entries")
not_in_newest_version = set(El_Salvador_no_duplicates) - ElSalvador_20210730_SET
print(f"There are {len(not_in_newest_version)} files in the old version that are not in the new version")
not_in_oldest_version =  ElSalvador_20210730_SET - set(El_Salvador_no_duplicates)
print(f"There are {len(not_in_oldest_version)} files in the new version that are not in the old version")
print(f"There are {len(set(El_Salvador_no_duplicates) | ElSalvador_20210730_SET)} unique ids in the union of the files")

There are 460 items in the old file and 271 in the new one
There are 0 duplicates in the new version
The difference between the first scraping and the last version is of 189 entries
There are 236 files in the old version that are not in the new version
There are 47 files in the new version that are not in the old version
There are 507 unique ids in the union of the files


In [69]:
# Going to merge the two files in one. The fields were not saved in the sae order so it is necessary to rearrange them

file_position = 0
with open(path + onlyfiles[file_position], 'r', encoding = "Latin1") as f:
    reader = csv.reader(f)
    ElSalvador_20210730 = list(reader)

file_position = 10
with open(path + onlyfiles[file_position], 'r') as f:
    reader = csv.reader(f)
    El_Salvador_no_duplicates = list(reader)
    
final_list = []
ids = {}
for item in El_Salvador_no_duplicates:
    if item[7] not in ids:
        final_item = []
        final_item.append(item[7])
        final_item.append('El Salvador')
        final_item.append('Diario Oficial')
        final_item.append(item[1])
        final_item.append(item[4])
        final_item.append(item[2])
        final_item.append(item[6])
        final_item.append(item[5])
        final_item.append(item[3])
        final_list.append(final_item)
        ids[item[7]] = 0
    
for item in ElSalvador_20210730:
    if item[3] not in ids:
        final_item = []
        final_item.append(item[3])
        final_item.append(item[1])
        final_item.append(item[2])
        final_item.append(item[0])
        final_item.append(item[7])
        final_item.append(item[6])
        final_item.append(item[10])
        final_item.append(item[9])
        final_item.append(item[4])
        final_list.append(final_item)
        ids[item[3]] = 0

In [81]:
# Checking the differences with bucket files

language = "spanish"
in_prefix = f"{language}_documents/HSSC/sentences/"
ids_S3 = {}
for i, obj in enumerate(s3.Bucket(s3BucketName).objects.all().filter(Prefix = in_prefix)):
    if ".json" in obj.key and "ing" not in obj.key:
        name = obj.key.split("/")[-1].split("_")[0]
        ids_S3[name] = 0
        

In [84]:
print(f"The number of scraped items that are not in sentences is {len(ids.keys() - ids_S3.keys())}")

The number of scraped items that are not in sentences is 236


In [98]:
# Here we go and fetch the missing files and put them in the raw_pdf_updated folder
prefix = f"{language}_documents/raw_pdf_updated/"


counter = 0
bucket = s3.Bucket(s3BucketName)
for item in final_list:
    if item[0] not in ids_S3:
        counter += 1
        key = prefix + item[0] + ".pdf"
        with requests.get(item[8], stream=True) as r:
            bucket.upload_fileobj(r.raw, key)
print(counter)

236
